## Setup

In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiLineString, LineString,MultiPolygon, MultiPoint
os.chdir('/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/Data')
pd.options.display.max_columns = None

/var/folders/p3/gs65vvgs1wd90mnxb3q3hvy80000gp/T/ipykernel_12702/1805498532.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Load Data
all_flowlines_gdf = gpd.read_file('all_flowlines.geojson')
matched_spills_gdf = gpd.read_file('matched_spills.geojson')

In [4]:
# Check size
print(all_flowlines_gdf.shape)
print(matched_spills_gdf.shape)

(335177, 26)
(417, 14)


In [5]:
all_flowlines_gdf.head(2)

,Operator,Fluid,Material,Diam_in,Status,Length_ft,SHAPE_Length,LOCATION_ID,FLOWLINEID,STARTLOCATIONID,FLOWLINEACTION,ENTIRELINEREMOVED,ACTIONDESCRIPTION,RECEIVE_DATE,OPERATOR_NUM,COMPANY_NAME,LOCATIONTYPE,ENDLAT,ENDLONG,STARTLAT,STARTLONG,PIPEMATERIAL,BEDDINGMATERIAL,TYPEOFFLUIDTRANS,MAXOPPRESSURE,geometry
0,EVERGREEN NATURAL RESOURCES LLC,Gas,polly,4.0,Active,2277.71,693.972162,NaN,NaN,NaN,None,None,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,NaN,"MULTILINESTRING ((524642.670 4117088.796, 5246..."
1,NOBLE ENERGY INC,Multiphase,Carbon Steel,3.0,Abandoned,651.58,198.525215,NaN,NaN,NaN,None,None,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,NaN,"MULTILINESTRING ((527997.281 4463899.920, 5281..."


In [6]:
matched_spills_gdf.head(2)

,trkg_num,Operator Name,facility_type,Spill_Desc,Spill Type,Root Cause,Preventative Measure,Root Cause Type,Detailed Root Cause Type,Long,Lat,facility_status,Metallic?,geometry
0,401576719,NOBLE ENERGY INC,TANK BATTERY,During decommisioning of facilitiy soil impact...,1,Historical impacts were discovered during dism...,The site was dismantled and reclamation has oc...,Unknown,Unknown,-104.602541,40.319666,CL,Unknown,POINT (533768.292 4463313.786)
1,401585074,NOBLE ENERGY INC,TANK BATTERY,During reclamation activities historical impac...,1,During reclamation of the oil and gas facility...,The location was plugged and abandoned and exc...,Unknown,Unknown,-104.754486,40.484583,CL,Unknown,POINT (520808.120 4481572.394)


In [8]:
# Check if CRS is the same for both files
if all_flowlines_gdf.crs != matched_spills_gdf.crs:
    flowlines_gdf = all_flowlines_gdf.to_crs(matched_spills_gdf.crs)

## Spatial Join

In [11]:
# Spatial join to find intersecting geometries
# Add attributes from flowlines to matching entries in crude_oil_gdf
complete_data = gpd.sjoin(all_flowlines_gdf, matched_spills_gdf, how="inner", op='intersects')
complete_data.shape

/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3490: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


(0, 40)

In [10]:
# Normalize the text in 'COMPANY_NAME' and 'Operator' columns before comparison
# This involves converting the text to lower case and stripping any leading/trailing spaces
validated_matches = complete_data[
    complete_data['COMPANY_NAME'].str.lower().str.strip() == complete_data['Operator'].str.lower().str.strip()
]

# Now, you can check the shape to see if there are any matches
print(validated_matches.shape)

(0, 40)
